PART I: Extracting the data

In [ ]:
import json
import pandas as pd
from ast import literal_eval
import glob
import csv
import copy
import os

Load data from fsm_data folder. Currently, the manual analysis do have the game DisYouCatchTheBall, TheDice, and Tangram so we remove them

> Note: the dir variable may differ in the local folder



In [ ]:
data = []
dir = '/content/fsm_data/*'
files = sorted(glob.glob(dir, recursive=False))
for single_file in files:
  if ((single_file=='/content/fsm_data/DidYouCatchTheBall_US.json') or (single_file == '/content/fsm_data/TheDice_US.json') or (single_file == '/content/fsm_data/TangramsRace.json')):
    files.remove(single_file)
  # if (single_file=='/content/fsm_data/TheDice_US.json'):
  #   files.remove(single_file)
  print(single_file)
#print(files)
for single_file in files:
    with open(single_file, 'r') as f:
        json_file = json.load(f)
        new_string = json.dumps(json_file, indent = 2)
        # print(new_string)
        data.append(json_file)

/content/fsm_data/Hackers_IN.json
/content/fsm_data/Indianboys_IN.json
/content/fsm_data/Indiangirls_IN.json
/content/fsm_data/MathParkour_US.json
/content/fsm_data/Raistar_IN.json
/content/fsm_data/fireballs20-us.json


Extract all of the text in the game. The considered text are from 'displayText' attribute in the json file

In [ ]:
c = []
for single_file in data:
    temp = single_file['states']
    d = []
    #print('------------------------------')
    for i in temp:
        att = list(i.keys())
        for key in att:
            if (key == 'displayText'):
                s = ''
                text = list(i[key].keys())
                for j in text:
                    s += i[key][j]
                if (not (s == '')):
                    d.append(s)
    c.append(d)

Extracting the data from csv file in manual_data folder, and turn them into pandas DataFrame


> Note: All csv need to be in the same name as the json file to keep the proper order (the right input goes with the right output). Furthermore, the number of files in both input and output must be the same (cannot have more input or have more output than the other)

We also drop some of the unneccessary columns (criterias that seems unrelated to NLP)



In [ ]:
dir_csv = '/content/manual_data/*'
csv_files = sorted(glob.glob(dir_csv, recursive=False))
df_list = (pd.read_csv(file) for file in csv_files)
labels = list(df_list)
for y in labels:
   y.drop(columns=["NAME","Day","Targeted Grade Level","Presence of Teams", "Team Dynamics No Teams", "Team Size", "Number of Teams", "Team Dynamics Between Teams", "Team Dynamics Within Teams", "Drawing Components [Rules]", "Drawing Components [Physical Objects]", "Drawing Components [Physical Space]", "Drawing Components [Timing]", "Drawing Components [Physicality]", "FSMD Components [Rules]", "FSMD Components [Physical Objects]", "FSMD Components [Physical Space]", "FSMD Components [Timing]", "FSMD Components [Physicality]", "Presence of Finite State Machine Diagram", "Output State Representation", "Transition State Representation", "Finite State Machine Diagram Consistency with Specified Rules", "State Consistency (Boxes)", "Transition Consistency (Arrows)", "Finite State Machine Diagram Completion", "States/Boxes", "Transitions/Arrows", "Numbered States", "Evidence of Programming Language Knowledge [Arrow(s) that loop to a previous state]"], inplace = True)


Keep the input and output as a single DataFrame (may be deleted later)

In [ ]:
li = []
for names in csv_files:
  dft = pd.read_csv(names, index_col=None, header=0)
  dft.drop(columns=["NAME","Day","Targeted Grade Level","Presence of Teams", "Team Dynamics No Teams", "Team Size", "Number of Teams", "Team Dynamics Between Teams", "Team Dynamics Within Teams", "Drawing Components [Rules]", "Drawing Components [Physical Objects]", "Drawing Components [Physical Space]", "Drawing Components [Timing]", "Drawing Components [Physicality]", "FSMD Components [Rules]", "FSMD Components [Physical Objects]", "FSMD Components [Physical Space]", "FSMD Components [Timing]", "FSMD Components [Physicality]", "Presence of Finite State Machine Diagram", "Output State Representation", "Transition State Representation", "Finite State Machine Diagram Consistency with Specified Rules", "State Consistency (Boxes)", "Transition Consistency (Arrows)", "Finite State Machine Diagram Completion", "States/Boxes", "Transitions/Arrows", "Numbered States", "Evidence of Programming Language Knowledge [Arrow(s) that loop to a previous state]"], inplace = True)
  li.append(dft)
frame = pd.concat(li, axis=0, ignore_index=True)
x = []
for col in frame.columns:
  if (col != 'Input'):
    x.append(col)

print(x)

['Game Descriptor', 'Content [Counting and Cardinality ]', 'Content [Operations and Algebraic Thinking ]', 'Content [Number and Operations in Base Ten]', 'Content [Number and Operations with Fractions]', 'Content [Measurement and Data]', 'Content [Geometry]', 'Content [Ratio and Proportions]', 'Content [The Number System]', 'Content [Expressions and Equations]', 'Content [Functions]', 'Content [Statistics and Probability ]', 'Progressive Levels', 'Content Adaptability', 'Game Facilitator', 'End-Goal', 'Technological Incorporation', 'Technological Dependency', 'Player Competition (No Teams)', 'Player Collaboration', 'Team Competition', 'Team Collaboration', 'Facilitator Competition', 'Facilitator Collaboration', 'Physicality', 'Physicality Option', 'Sweat Factor', 'Physical Contact', 'Style of Physical Contact', 'Physical Space Diagram', 'Physical Environment', 'If you selected 0 (Unspecified), select one of the following codes related to the size of the environment for gameplay based o

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel

Initializing the size for our input to feed to the model

In [ ]:
MAX_LEN = 133
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2 
EPOCHS = 3
LEARNING_RATE = 1e-05
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

cuda:0


The following two cells create a single DataFrame that holds the input text and the labels

In [ ]:
input = []
for arr in c:
  temp = ''
  for i in arr:
    i+='.'
    temp += i
  input.append(temp)

In [ ]:
tempf = []
for u in labels:
  tempt = []
  for k in x:
    t = (u[k].values)[0]
    tempt.append(t)
  tempf.append(tempt)

dataf = {'Input': input, 'Output': tempf}
train_data = pd.DataFrame(dataf)
# print(df)

PART II: Building the model

We define a PyTorch Dataset class called MultiLabelDataset that is used to preprocess text data for multi-label text classification tasks using the DistilBERT model.

We put our DataFrame into the class and it will tokenize the text (in a way needed for BERT), generate the attention mask and put all of them in a Tensor object. 

More info about the tokenization process: BERT needs the input to be breaks down into smaller tokens and padd all inputs to be the same length (also, all sentences must be padded with the '[CLS]' and '[SEP]' tokens at start and end)

In [ ]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len, new_data=False):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.Input
        self.new_data = new_data
        
        if not new_data:
            self.targets = self.data.Output
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        out = {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        }
        
        if not self.new_data:
            out['targets'] = torch.tensor(self.targets[index], dtype=torch.float)

        return out

Split out data into training set and validation set

In [ ]:
train_size = 0.7

train_df = train_data.sample(frac=train_size, random_state=123)
val_df = train_data.drop(train_df.index).reset_index(drop=True)
train_df = train_df.reset_index(drop=True)


print("Orig Dataset: {}".format(train_data.shape))
print("Training Dataset: {}".format(train_df.shape))
print("Validation Dataset: {}".format(val_df.shape))

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)
training_set = MultiLabelDataset(train_df, tokenizer, MAX_LEN)
val_set = MultiLabelDataset(val_df, tokenizer, MAX_LEN)

Orig Dataset: (6, 2)
Training Dataset: (4, 2)
Validation Dataset: (2, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 8
                }

val_params = {'batch_size': VALID_BATCH_SIZE,
               'shuffle': False,
               'num_workers': 8
                }

training_loader = DataLoader(training_set, **train_params)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


This is a model that is built up on DistilBertModel, which is a lighter, faster version of the normal Bert Model.

When calling forward(), the model breaks down the inputs into hidden states, which refer to the internal representation of a sequence of text, such as a sentence or a document, that is learned by a machine learning model

In the case of the DistilBERT model, the hidden state refers to the internal representation of the input text at each layer of the model. Each layer of the model takes the output of the previous layer and produces a new hidden state that captures increasingly complex and abstract features of the input text. The final hidden state of the last layer, corresponding to the [CLS] token in the input sequence, is typically used as the input to downstream tasks such as text classification, question answering, or text generation.

After we have the hidden states, the model run the classifer, which is a sequence of three layers. The last layer would be a fully connected layer with 768 input neurons and 54 output neurons (54 represents our classes). The output of this classifier will be used to predict the classification label of the input text.

Having multiple layers of transformers allows the model to capture increasingly complex and abstract features of the input text, as each layer can build on the representations learned by the previous layer. However, increasing the number of layers can also make the model more prone to overfitting, as it may start to memorize the training data instead of learning general patterns.

In [ ]:
class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(768, 768),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(768, 54)
        )

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        out = hidden_state[:, 0]
        out = self.classifier(out)
        return out

model = DistilBERTClass()
model.to(DEVICE);

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The optimizer is an algorithm used during the training of a neural network to adjust the model's weights and biases based on the computed gradients of the loss function. The goal of the optimizer is to minimize the loss function and improve the accuracy of the model's predictions.
The Adam optimizer is a popular optimization algorithm that is commonly used in deep learning. It is an extension of stochastic gradient descent (SGD) and is known for its ability to converge quickly and efficiently, even for large and complex models.



In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

During one epoch, the model receives the entire training dataset, processes it forward and backward through the network, and updates the model parameters. 

The train function is called for each epoch and sets the model to training mode using model.train(). It then iterates over the batches in the training_loader, loads the batch data onto the DEVICE, and passes it through the model to obtain the outputs.

The optimizer's gradients are set to zero with optimizer.zero_grad() to prevent accumulation of gradients from previous batches. The loss is computed using the binary_cross_entropy_with_logits function from torch.nn.functional. This function computes the binary cross-entropy loss between the outputs and the targets.

loss.backward() computes the gradients of the binary cross-entropy loss with respect to each parameter of the model, which is then used by the optimizer to update the model parameters in the next step of the training loop

In [ ]:

def train(epoch):
    model.train()
    for _, data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(DEVICE, dtype=torch.long)
        mask = data['mask'].to(DEVICE, dtype=torch.long)
        token_type_ids = data['token_type_ids'].to(DEVICE, dtype=torch.long)
        targets = data['targets'].to(DEVICE, dtype=torch.float)

        outputs = model(ids, mask, token_type_ids)

        print(type(outputs))
        optimizer.zero_grad()
        loss = torch.nn.functional.binary_cross_entropy_with_logits(outputs, targets)
        
        if _ % 5000 == 0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        loss.backward()
        optimizer.step()

for epoch in range(EPOCHS):
    train(epoch)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Ber

         0         1         2         3         4         5         6   \
0  0.167245 -0.002925  0.052576 -0.039000 -0.217555 -0.149561 -0.125101   
1  0.116708  0.073087  0.044703 -0.133143 -0.300710 -0.054160 -0.083151   
2  0.242028 -0.031560  0.059711 -0.043476 -0.142392 -0.128237 -0.123465   
3  0.089968  0.057981  0.002981 -0.056353 -0.157701 -0.192990 -0.154423   

         7         8         9   ...        44        45        46        47  \
0  0.067683 -0.029428 -0.075443  ...  0.042194 -0.153113 -0.046316 -0.172097   
1  0.024772  0.009650 -0.093608  ...  0.021583 -0.140596 -0.164997 -0.145313   
2  0.081921 -0.019117 -0.038668  ...  0.021207 -0.172215 -0.113014 -0.157280   
3  0.016763 -0.059301 -0.004641  ...  0.065751 -0.154059 -0.068631 -0.107838   

         48        49        50        51        52        53  
0  0.103648 -0.130676 -0.188862 -0.171284 -0.347703 -0.091149  
1  0.166138 -0.114653 -0.126334 -0.005592 -0.410020  0.054568  
2  0.072922 -0.004060 -0.209105

1it [00:00,  2.95it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the 

         0         1         2         3         4         5         6   \
0  0.173767  0.012844  0.098920 -0.150758 -0.162595 -0.101925 -0.064435   
1  0.149292 -0.013768  0.011610 -0.004073 -0.218571 -0.231052 -0.127280   
2  0.242110 -0.030672  0.097750 -0.097538 -0.187477 -0.177646 -0.195789   
3  0.150232  0.015323  0.006123 -0.078625 -0.225446 -0.200146 -0.100145   

         7         8         9   ...        44        45        46        47  \
0 -0.062807 -0.036178 -0.150250  ... -0.047056 -0.154401 -0.107743 -0.211837   
1  0.048084 -0.023636 -0.084689  ... -0.008184 -0.183350 -0.087624 -0.170612   
2 -0.049693  0.043677 -0.064663  ... -0.018161 -0.169161 -0.125703 -0.145265   
3 -0.118565 -0.059573 -0.075705  ... -0.029309 -0.142144 -0.136959 -0.129792   

         48        49        50        51        52        53  
0  0.060329 -0.059799 -0.054053 -0.077098 -0.448194  0.023318  
1  0.104372 -0.102476 -0.153862 -0.210152 -0.389625 -0.031330  
2  0.090830 -0.076960 -0.097272

1it [00:00,  2.96it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the 

         0         1         2         3         4         5         6   \
0  0.310647 -0.059310  0.043037 -0.073052 -0.176255 -0.189937 -0.100302   
1  0.249003 -0.032789  0.053527 -0.001169 -0.262397 -0.136251 -0.167822   
2  0.214111  0.017836  0.143347 -0.111597 -0.079665 -0.028124 -0.120370   
3  0.072970 -0.045385 -0.054866 -0.017480 -0.235257 -0.101760 -0.189158   

         7         8         9   ...        44        45        46        47  \
0  0.011870 -0.050112  0.006446  ... -0.034575 -0.158069 -0.093821 -0.200746   
1  0.055742 -0.000586 -0.083491  ...  0.072664 -0.116734 -0.094908 -0.263287   
2 -0.080208 -0.034351 -0.121334  ... -0.039175 -0.120834 -0.224620 -0.296402   
3 -0.018664  0.005028 -0.016451  ... -0.039547 -0.106742 -0.114854 -0.121749   

         48        49        50        51        52        53  
0  0.081225 -0.042399 -0.195142 -0.102207 -0.309283 -0.168804  
1  0.044863 -0.053491 -0.192234 -0.240554 -0.326862 -0.092431  
2  0.052029 -0.049742 -0.063912

1it [00:00,  3.16it/s]


In [ ]:
val_loader = DataLoader(val_set, **val_params)

all_test_pred = []

def test(epoch):
    model.eval()
    
    with torch.inference_mode():
    
        for _, data in tqdm(enumerate(val_loader, 0)):


            ids = data['ids'].to(DEVICE, dtype=torch.long)
            mask = data['mask'].to(DEVICE, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(DEVICE, dtype=torch.long)
            outputs = model(ids, mask, token_type_ids)
            print(outputs)
            probas = torch.sigmoid(outputs)

            all_test_pred.append(probas)
            
            
    return probas
probas = test(model)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Ber

tensor([[ 0.1151,  0.0179, -0.0501,  0.0319, -0.2003, -0.1207, -0.0916,  0.0145,
         -0.0053, -0.0379, -0.2164, -0.1339, -0.0189,  0.1045, -0.0658, -0.1487,
          0.0671,  0.0379,  0.0711,  0.1347, -0.0142, -0.1379, -0.0948,  0.0114,
         -0.0422, -0.0923,  0.1147, -0.1087, -0.0751,  0.1036,  0.1045, -0.0578,
          0.1134, -0.0248, -0.0070, -0.0179, -0.0833, -0.0551,  0.0804, -0.0269,
          0.1492, -0.0734, -0.0152,  0.0035,  0.1317, -0.1251, -0.0224, -0.1231,
          0.0606, -0.0615, -0.0901,  0.0022, -0.3573,  0.0954],
        [ 0.0751,  0.0340, -0.0838, -0.0112, -0.1957, -0.1010, -0.0939, -0.0012,
          0.0108, -0.0389, -0.0994, -0.1312, -0.0339,  0.0478, -0.0800, -0.1084,
          0.0838,  0.0274,  0.0077,  0.0857, -0.0191, -0.1389, -0.1118,  0.0224,
         -0.1075, -0.1288,  0.1357, -0.0998, -0.0276,  0.0989,  0.0697, -0.0571,
          0.1234, -0.0492,  0.0134,  0.0143, -0.0493, -0.0539,  0.0029, -0.0135,
          0.1454, -0.0659, -0.0463,  0.0048, 

1it [00:00,  3.24it/s]


In [ ]:
TEST_BATCH_SIZE = 6

test_params = {'batch_size': TEST_BATCH_SIZE,
               'shuffle': False,
               'num_workers': 8
                }


test_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
test_loader = DataLoader(test_set, **test_params)


temp_df = pd.DataFrame()
def test(epoch):
    model.eval()
    
    with torch.inference_mode():
    
        for _, data in tqdm(enumerate(test_loader, 0)):


            ids = data['ids'].to(DEVICE, dtype=torch.long)
            mask = data['mask'].to(DEVICE, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(DEVICE, dtype=torch.long)
            outputs = model(ids, mask, token_type_ids)


            temp_df = pd.DataFrame(outputs.cpu().detach().numpy())
            #df_rounded = temp_df.applymap(lambda x: 1 if x > 0.5 else 0)
            temp_df.to_csv("/content/results.csv", index=False)
            print(temp_df)

            probas = torch.sigmoid(outputs)

            all_test_pred.append(probas)
            
            
    return probas
probas = test(model)

temp_df.to_csv("results.csv", index=False)
temp_df.to_csv("/content/results.csv", index=False)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-p

         0         1         2         3         4         5         6   \
0  0.240727 -0.086226  0.063631 -0.054719 -0.244915 -0.171508 -0.207122   
1  0.150497  0.050478  0.114338 -0.174330 -0.238030 -0.058276 -0.118511   
2  0.144219 -0.000399  0.031171 -0.058357 -0.261946 -0.130993 -0.166629   
3  0.092364 -0.035548  0.045001 -0.106918 -0.210249 -0.134337 -0.200580   
4  0.237576 -0.054163  0.072381 -0.081666 -0.241144 -0.178020 -0.173918   
5  0.111617 -0.006596 -0.014167 -0.098474 -0.245513 -0.109415 -0.163745   

         7         8         9   ...        44        45        46        47  \
0  0.069259 -0.027271 -0.071921  ...  0.006373 -0.182811 -0.126584 -0.245638   
1 -0.005190 -0.042398 -0.142083  ... -0.036281 -0.187027 -0.161045 -0.191858   
2 -0.021197 -0.018849 -0.088459  ...  0.054625 -0.180659 -0.086190 -0.183680   
3 -0.052968 -0.023912  0.004654  ... -0.042078 -0.182569 -0.117808 -0.198894   
4  0.026863 -0.022746 -0.075500  ... -0.024170 -0.180087 -0.083462 -0.2321